In [1]:
pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 62.5 MB/s 
     |████████████████████████████████| 895 kB 71.6 MB/s 
     |████████████████████████████████| 596 kB 46.9 MB/s 
     |████████████████████████████████| 59 kB 6.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request


In [3]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)



In [4]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary
task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)



Downloading:   0%|          | 0.00/779 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [5]:
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]



In [6]:
labels

['anger', 'joy', 'optimism', 'sadness']

In [9]:
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [10]:
text = "Celebrating my promotion 😎"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

array([0.01446436, 0.938211  , 0.03615636, 0.01116814], dtype=float32)

In [11]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) joy 0.9382
2) optimism 0.0362
3) anger 0.0145
4) sadness 0.0112


In [12]:
def EmojiSA(text) : 
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [16]:
EmojiSA('I`m sad  😖  😖  😖 ')

1) sadness 0.9842
2) joy 0.0073
3) anger 0.005
4) optimism 0.0035


In [15]:
EmojiSA('I`m happy  😖  😖  😖 ')

1) joy 0.7872
2) sadness 0.183
3) optimism 0.0203
4) anger 0.0096


In [17]:
EmojiSA('I got my result  😖  😖  😖 ')

1) sadness 0.9238
2) anger 0.0557
3) joy 0.0141
4) optimism 0.0064


In [18]:
 EmojiSA('I got my result  🥳🥳🥳')

1) joy 0.9479
2) optimism 0.026
3) sadness 0.0131
4) anger 0.0129


In [19]:
 EmojiSA('I got my result  🤬🤬🤬')

1) anger 0.6644
2) sadness 0.16
3) joy 0.1586
4) optimism 0.0171


In [20]:
 EmojiSA('I got my result   🤪  🤪  🤪 ')

1) joy 0.9488
2) optimism 0.0254
3) anger 0.016
4) sadness 0.0098
